In [18]:
from datasets import load_dataset
import json

In [2]:
ds = load_dataset("habanoz/finetune_mix_v1")

Generating train split:   0%|          | 0/28530 [00:00<?, ? examples/s]

In [3]:
from transformers import AutoTokenizer

In [4]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-8B")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [38]:
ds_sub = ds["train"].take(2)

In [39]:
ds_sub

Dataset({
    features: ['conversations', 'tools'],
    num_rows: 2
})

In [74]:
def formatting_prompts_func(example):
    convos = [msg|{'tool_calls':json.loads(msg['tool_calls']) if msg['tool_calls'] else None } for msg in example["conversations"]]
    tools = json.loads(example["tools"])
    text = tokenizer.apply_chat_template(convos, tokenize = False, add_generation_prompt = False, tools=tools)
    print(text)
    return { "text" : text }

In [75]:
ds_sub_new = ds_sub.map(formatting_prompts_func, batched=False, batch_size=4)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

<|im_start|>user
Can brain cells move? By movement I mean long distance migration (preferably within the brain only).<|im_end|>
<|im_start|>assistant
<think>

</think>

The question is relatively broad and one should take into account that the brain not only consists of neurons, but also glial cells (supportive cells) and pre-mitotic neuronal stem cells. Furthermore, as critical fellow-scientists have indicated, developmental stage is very important, as the developing embryonic brain is very different from the adult brain.
However, after sifting through various publications, the answer to the question is actually remarkably simple: Yes, brain cells migrate.
In  the adult brain glial cells migrate in the brain (Klämbt, 2009). Glial cells are involved in a myriad of functions, but a notable example of migrating glial cells are the oligodendrocytes that migrate relative long distances to find their target axons onto which they wrap themselves to form the insulating myelin sheath (Tsai and

In [70]:
ds_sub_new

Dataset({
    features: ['conversations', 'tools', 'text'],
    num_rows: 2
})

In [62]:
ds_sub_new['text'][0]

'<|im_start|>user\nCan brain cells move? By movement I mean long distance migration (preferably within the brain only).<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\nThe question is relatively broad and one should take into account that the brain not only consists of neurons, but also glial cells (supportive cells) and pre-mitotic neuronal stem cells. Furthermore, as critical fellow-scientists have indicated, developmental stage is very important, as the developing embryonic brain is very different from the adult brain.\nHowever, after sifting through various publications, the answer to the question is actually remarkably simple: Yes, brain cells migrate.\nIn  the adult brain glial cells migrate in the brain (Klämbt, 2009). Glial cells are involved in a myriad of functions, but a notable example of migrating glial cells are the oligodendrocytes that migrate relative long distances to find their target axons onto which they wrap themselves to form the insulating myelin sheath

In [67]:
ds_sub_new['text'][0]

['<|im_start|>user\nCan brain cells move? By movement I mean long distance migration (preferably within the brain only).<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\nThe question is relatively broad and one should take into account that the brain not only consists of neurons, but also glial cells (supportive cells) and pre-mitotic neuronal stem cells. Furthermore, as critical fellow-scientists have indicated, developmental stage is very important, as the developing embryonic brain is very different from the adult brain.\nHowever, after sifting through various publications, the answer to the question is actually remarkably simple: Yes, brain cells migrate.\nIn  the adult brain glial cells migrate in the brain (Klämbt, 2009). Glial cells are involved in a myriad of functions, but a notable example of migrating glial cells are the oligodendrocytes that migrate relative long distances to find their target axons onto which they wrap themselves to form the insulating myelin sheat

In [51]:
ds_sub_tools = ds["train"].filter(lambda x: x['tools']!='null').take(2)

In [52]:
ds_sub_tools[0]

{'conversations': [{'content': "Tokyo'da gezilecek en iyi 7 yeri öğrenebilir miyim?",
   'role': 'user',
   'tool_calls': None},
  {'content': '',
   'role': 'assistant',
   'tool_calls': '[{"name": "list_attractions", "arguments": {"city": "Tokyo", "limit": 7}}]'}],
 'tools': '[{"name": "list_attractions", "description": "List top attractions for the given city. Returns an array of attraction objects (name, address, shortDescription, rating, category).", "parameters": {"type": "object", "properties": {"city": {"type": "string", "description": "City name or query (e.g., \'Paris, France\' or \'Seattle, WA\')."}, "limit": {"type": "integer", "description": "Maximum number of attractions to return.", "minimum": 1, "default": 10}}, "required": ["city"]}}, {"name": "search_hotel", "description": "Search for hotels for the given check-in/check-out dates and number of guests. Returns an array of hotel objects (name, address, rating, availableRooms, price {amount, currency}, bookingUrl). Dates

In [53]:
ds_sub_tools.map(formatting_prompts_func, batched=False, batch_size=4)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

[{'content': "Tokyo'da gezilecek en iyi 7 yeri öğrenebilir miyim?", 'role': 'user', 'tool_calls': None}, {'content': '', 'role': 'assistant', 'tool_calls': [{'name': 'list_attractions', 'arguments': {'city': 'Tokyo', 'limit': 7}}]}]
[{'name': 'list_attractions', 'description': 'List top attractions for the given city. Returns an array of attraction objects (name, address, shortDescription, rating, category).', 'parameters': {'type': 'object', 'properties': {'city': {'type': 'string', 'description': "City name or query (e.g., 'Paris, France' or 'Seattle, WA')."}, 'limit': {'type': 'integer', 'description': 'Maximum number of attractions to return.', 'minimum': 1, 'default': 10}}, 'required': ['city']}}, {'name': 'search_hotel', 'description': 'Search for hotels for the given check-in/check-out dates and number of guests. Returns an array of hotel objects (name, address, rating, availableRooms, price {amount, currency}, bookingUrl). Dates should be ISO 8601 (YYYY-MM-DD).', 'parameters': 

Dataset({
    features: ['conversations', 'tools', 'text'],
    num_rows: 2
})